In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import tensorflow.keras as keras
import pandas as pd
import os

##making the list of wav files

In [7]:
import glob
path = r'/content/drive/MyDrive/Respiratory sound analysis, classification and disease detection (Final Year Design Project)/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/*.wav'
files = glob.glob(path)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##testing audio from list


In [ ]:
from pydub import AudioSegment
from pydub.playback import play
 
song = AudioSegment.from_wav(files[100])
print('playing sound using  pydub')
play(song)

playing sound using  pydub


In [4]:
import librosa

In [13]:
f = files[101]

sound, sample_rate = librosa.load(f)
stft = np.abs(librosa.stft(sound)) 

zcr = np.mean(librosa.feature.zero_crossing_rate(sound, sample_rate),axis=1)

rms = np.mean(librosa.feature.rms(sound, sample_rate),axis=1)

In [14]:
np.mean(zcr)

0.0035896059893408255

##this function returns numPy ndArray of mfcc, chroma, mel spectrogram values, in one array

In [15]:
def sendFeature(f):
  sound, sample_rate = librosa.load(f)
  stft = np.abs(librosa.stft(sound))  
    
  mfccs = np.mean(librosa.feature.mfcc(y=sound, sr=sample_rate, n_mfcc=40),axis=1)
  #chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate),axis=1)
  mel = np.mean(librosa.feature.melspectrogram(sound, sr=sample_rate),axis=1)
  zcr = np.mean(librosa.feature.zero_crossing_rate(sound, sample_rate),axis=1)
  
  rms = np.mean(librosa.feature.rms(sound, sample_rate),axis=1)


  pname = os.path.basename(f).split("_")
  SL = pname[0]
  row = []
  
  row = np.concatenate((mfccs,mel, zcr))

  row = np.insert(row, 0, SL)
  return row


##creating dataframe of mfcc, chroma, mel 1D values without class column

In [16]:
df = pd.DataFrame()

for x in files:
  Row = sendFeature(x)
  df = df.append(pd.DataFrame(data=[Row]))

##making dataframe of diagnosis data

In [17]:
classes = pd.read_csv("/content/drive/MyDrive/Respiratory sound analysis, classification and disease detection (Final Year Design Project)/Respiratory_Sound_Database/Respiratory_Sound_Database/patient_diagnosis.csv")

In [18]:
df[0].value_counts()

130.0    66
151.0    28
107.0    28
138.0    27
172.0    27
         ..
125.0     1
126.0     1
128.0     1
123.0     1
184.0     1
Name: 0, Length: 126, dtype: int64

In [19]:
resultPath = "/content/drive/MyDrive/Respiratory sound analysis, classification and disease detection (Final Year Design Project)/Respiratory_Sound_Database/Respiratory_Sound_Database/patient_diagnosis.csv"

In [20]:
title = ["SL", "class"]

diag = pd.read_csv(resultPath, names=title)

In [21]:
df.rename(columns = {0:'SL'}, inplace = True)

In [22]:
df2 = pd.merge(df, diag, on="SL")

###merging dataframes of attributes and label column

In [23]:
df2 = df2.drop('SL', axis=1)

In [24]:
df2

,1,2,3,4,5,6,7,8,9,10,...,161,162,163,164,165,166,167,168,169,class
0,-376.834167,112.070412,54.905434,48.952473,31.104082,26.692186,17.166862,15.440213,9.706236,9.781637,...,0.000047,0.000044,0.000041,0.000038,0.000024,7.245791e-06,6.603987e-07,1.131897e-08,0.005526,COPD
1,-355.735504,112.486313,64.843376,33.597191,25.152960,25.082975,23.064644,19.242466,16.606459,15.464816,...,0.000027,0.000026,0.000025,0.000022,0.000015,4.328226e-06,4.023240e-07,5.512888e-08,0.003274,COPD
2,-388.970917,105.133408,49.908825,44.876896,30.665163,29.289984,21.182224,20.246838,14.096419,14.026716,...,0.000043,0.000039,0.000037,0.000034,0.000021,6.409333e-06,5.948819e-07,5.328588e-08,0.004312,COPD
3,-353.922516,105.388161,66.852875,36.985683,26.500805,26.432636,25.833517,23.252085,20.827061,18.568815,...,0.000051,0.000048,0.000047,0.000042,0.000027,7.938614e-06,7.775130e-07,2.073103e-07,0.002623,COPD
4,-353.211151,111.863373,62.016300,30.860928,23.401262,24.418919,24.640217,22.140684,18.652845,15.788815,...,0.000025,0.000024,0.000023,0.000021,0.000013,3.950217e-06,3.776953e-07,5.037608e-08,0.003196,COPD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,-291.696014,178.602997,64.844208,20.630623,21.570168,13.506413,-3.262139,-3.864437,12.238049,22.694208,...,0.000012,0.000012,0.000011,0.000011,0.000007,1.917187e-06,1.739017e-07,4.180725e-09,0.008666,COPD
916,-502.216705,100.117805,60.128754,32.277088,36.383972,38.851910,28.873722,12.902280,10.237914,15.450107,...,0.000025,0.000014,0.000006,0.000011,0.000002,6.539217e-07,1.711861e-07,9.336559e-08,0.002331,Healthy
917,-524.256897,103.861298,58.770683,24.850790,34.807739,35.451801,24.114174,11.107311,8.224469,13.108119,...,0.000032,0.000018,0.000007,0.000015,0.000002,7.095514e-07,1.193747e-07,1.740210e-08,0.005799,Healthy
918,-515.420593,78.584358,59.590454,41.739243,40.581760,36.480518,25.066301,17.418379,14.219076,14.065955,...,0.000054,0.000032,0.000033,0.000041,0.000017,5.137932e-06,6.610316e-07,7.150481e-08,0.002766,Healthy


###train test, X Y split

In [45]:
from sklearn.model_selection import train_test_split

X =df2.loc[:,df2.columns != 'class']
Y = df2['class']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

In [46]:
from sklearn.preprocessing import MinMaxScaler

mc = MinMaxScaler()

mc.fit(X_train)
mc_trainx=mc.transform(X_train)
mc_testx= mc.transform(X_train)

In [ ]:
X_train.shape

(736, 169)

In [ ]:
mc_trainx.shape

(736, 169)

In [ ]:
y_test

319         COPD
377         COPD
538         COPD
296         COPD
531         COPD
         ...    
382         COPD
97          COPD
906         COPD
467    Pneumonia
732         COPD
Name: class, Length: 184, dtype: object

In [47]:
mc_trainx

array([[3.45171196e-01, 5.65145251e-01, 7.66245824e-01, ...,
        3.33574423e-05, 5.39598104e-05, 2.95501951e-02],
       [5.94763390e-01, 2.34400774e-01, 7.88176344e-01, ...,
        1.85655794e-04, 1.54333609e-03, 1.14134837e-02],
       [7.56083052e-01, 7.71946625e-01, 8.51997912e-01, ...,
        5.76484664e-04, 3.62775696e-04, 9.16845000e-02],
       ...,
       [4.40473643e-01, 3.31420066e-01, 6.10290270e-01, ...,
        2.37373036e-04, 7.19057007e-05, 3.62119819e-02],
       [6.51182687e-01, 4.64153800e-01, 7.47310665e-01, ...,
        9.89904831e-04, 4.57510822e-03, 2.70453431e-02],
       [6.02713586e-01, 3.93431210e-01, 5.17991474e-01, ...,
        1.09673481e-03, 3.70889992e-04, 5.21490031e-02]])

In [48]:
from sklearn.neighbors import KNeighborsClassifier  
classifier= KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2 )  
classifier.fit(mc_trainx, y_train)  

KNeighborsClassifier()

In [49]:
y_pred= classifier.predict(X_test)  

In [50]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.8478260869565217